# Information Retrieval - Document Ranking and Search
### Aryan Mehra  (2017A7PS0077P)  ||  Siddhant Khandelwal (2017A7PS0127P)

### Text Queries and Search 

We first import the necessary libraries and modules.

In [56]:
import nltk
import numpy as np
import math
import pickle
import sys
from bs4 import BeautifulSoup as bsoup
from spellchecker import SpellChecker
from nltk.stem import PorterStemmer

Setting appropriate options for display if need be - we would like to display full numpy arrays.

In [57]:
# For printing the whole npy array
np.set_printoptions(threshold=sys.maxsize)

### Spell correction Heuristic
This is the spell correction heuristic. We check if the word is misspelled and then replace it with the correct word before the actual search in the backend. The spell_correct function will return the spell corrected query.

In [58]:
def spell_correct(query):
    print("Spell Check is activated ........ Running Spell Check")
    spell = SpellChecker()
    misspelled = spell.unknown(query.split())
    if misspelled:
        for word in query.split():
            if word in misspelled:
                print("Correcting " + word + " to " + spell.correction(word))
                query = query.replace(word, spell.correction(word))
    return query

A simple utility to stem the token in case we use the related word search heuristic. 

In [59]:
def get_stemmed_token(token):
    porter = nltk.PorterStemmer()
    return porter.stem(token)

### Processing the query vector - Nearest word heuristic
The following function ***process_query_vector*** takes as input the tokenized query vector, vocabulary words and their iverse word-to-index mapping, and the document frequency of every term in the corpus along with ***"N"*** - the number of documents itself. The ***Spell Correction (Bonus Heuristic)*** is also present inside this function which can be easily commented outto notice the difference that it will create to misspelled queries.

***Steps -*** We first see whether the word is in the vocabulary or not. In case not, we give the user the option to replace the word with the highest occuring word of that "non existant" query term's stemmed equivalence class. We use the stemmed vocabulary metadata for the same. Hence, the ambiguity is resolved  in both single word and multi-term queries. We then proceed to do a ****LTC operation scheme*** on the query, using inverse document frequency and cosine normalization as well. Finally the processed query and vector is returned. 

In [60]:
def process_query_vector(query, vocabulary_keys, inverse_vocab_word_dict, term_document_frequency, N):
    query_text = ''
    for token in query:
        query_text = query_text + ' ' + str(token)
    query = query_text
    query = query.lower()
    
    
    print("Nearest word (Did you mean) heuristic is automatically activated ........... ")

    # ----------------  BONUS HEURISTIC ------------------------
    
    query = spell_correct(query)
    
    # ----------------  BOUNUS HEURISTIC ------------------------

    query_vector = np.zeros(len(vocabulary_keys))
    query = nltk.word_tokenize(query)

#     for one word queries we deal separately
    if(len(query) == 1 and query[0] not in inverse_vocab_word_dict):
        
        print(query[0] + " is not found in vocabulary. Using most appropriate substitution using root word analysis! ")
        stemmed_token = get_stemmed_token(query[0])

        # Now the query contains all tokens as it is, only the ones that do not excist
        # in the vocabulary are replaced by their stemmed root versions
        
        stemmed_vocab = pickle.load(open("stemmed_vocab.pkl", "rb"))
        if(stemmed_token not in stemmed_vocab):
            print("Sorry, Could not replace, no search results found, exiting the program by default")
            exit(0)
        
        fixed_token = stemmed_vocab[token][0]
        print("Did you mean " + fixed_token + "? Press y for yes: ")
        choice = input()
        if choice != 'y':
            print("Sorry, No search results found, exiting the program by default")
            exit(0)
        
        query[query.index(token)] = fixed_token
        # Query has the wrong word replaced by the most common rooted word (with the same root as the wrong word)
        print("Changed query: " + ' '.join(query))
        query_vector[inverse_vocab_word_dict[fixed_token]] = query_vector[inverse_vocab_word_dict[fixed_token]] + 1
        
        
    elif(len(query) >= 1):
        for token in query:
            if(token not in inverse_vocab_word_dict):
                print(token + " not found in vocabulary.")
                stemmed_token = get_stemmed_token(token)
                stemmed_vocab = pickle.load(open("stemmed_vocab.pkl", "rb"))
                if(stemmed_token not in stemmed_vocab):
                    continue
                fixed_token = stemmed_vocab[stemmed_token][0]
                print("Did you mean " + fixed_token + "? Press y for yes: ")
                choice = input()
                if choice != 'y':
                    print("Skipping " + token)
                    continue
                query[query.index(token)] = fixed_token
                print("Changed query: " + ' '.join(query))
                query_vector[inverse_vocab_word_dict[fixed_token]] = query_vector[inverse_vocab_word_dict[fixed_token]] + 1
            else:
                query_vector[inverse_vocab_word_dict[token]] = query_vector[inverse_vocab_word_dict[token]] + 1

    # processing log calculations (L)
    for i in range(query_vector.shape[0]):
        if(query_vector[i] > 0):
            query_vector[i] = 1 + math.log(query_vector[i])

    # processing term normalization (T)
    for i in range(query_vector.shape[0]):
        if(query_vector[i] == 0):
            continue
        query_vector[i] = query_vector[i] * math.log(N/term_document_frequency[i])

    # Cosine normalization (C)
    temp_query_vector = np.copy(query_vector)
    temp_query_vector = np.square(temp_query_vector)
    temp_query_vector = np.sum(temp_query_vector)
    temp_query_vector = np.sqrt(temp_query_vector)
    query_vector = np.divide(query_vector, temp_query_vector)

    return query, query_vector

### Score calculation with query vector and document vector
This function is also quite self explanatory. We simply calculate the dot product of the vectors to give the scores of the query with all the documents in the corpus. We skip and comment out the division with vector magnitudes because due to the LTC and LNC scheme the magnitudes are anyway equal to 1.

In [61]:
def calculate_score(query_vector, database_lnc):
    scores = []
    for id, document_vector in enumerate(database_lnc):
        score = np.dot(query_vector, document_vector)
        
#         dinominator1 = np.linalg.norm(query_vector)
#         dinominator2 = np.linalg.norm(document_vector)
#         score = score/dinominator1
#         score = score/dinominator2
        
        scores.append([id, score])
    return scores

### Heuristic for Title Weighting
This function encodes the heuristic of title weighting. The score is increased for the documents whose title contains words from the query. It is assumed that such documents will be more important within the fetched documents and otherwise as well. 

In [62]:
def title_weighting(scores, query, doc_titles):
    title_weight = 0.1
    trivial_words = ["of", "and", "a", "the", "an", "is"]
    
    print("Title weighting is activated ....... ")
    
    for doc_id, doc_title in enumerate(doc_titles):
        doc_title_temp = doc_title.lower()
        count = 0
        for word in query:
            if word in doc_title_temp:
                if(word not in trivial_words):
#                     print("The title weighted word is " + word + " for the title " + doc_title)
                    scores[doc_id][1] = scores[doc_id][1] + title_weight       
        
    return scores

### Scoring function and Results Display
This function has the main calls to the process_query_vector function and score calculation as well. The lines marked between comments can be used to activate and deactivate the title weight function as need be.

In [63]:
def scoring(query, database_lnc, vocabulary_keys, inverse_vocab_word_dict, term_document_frequency, N, doc_titles):
    corrected_query, query_vector = process_query_vector(query, vocabulary_keys, inverse_vocab_word_dict, term_document_frequency, N)
    scores = calculate_score(query_vector, database_lnc)
    
#     #------------------- title weighting heuristic -----------------------

    scores = title_weighting(scores, corrected_query , doc_titles)

#     #------------------- title weighting heuristic -----------------------
    
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    print("\nTop Scoring Documents are: ")
    for ind in range(10):
        if(scores[ind][1] == 0):
            break
        print(doc_titles[scores[ind][0]] + " is at rank " +
              str(ind+1) + " Score: " + str(scores[ind][1]))

### Loading and populating appropriate data structures and metadata.
We load the database that we had stored in the LNC scheme format in the index construction phase, along with the vocabulary dictionary. ***"N"*** represents the number of documents. Similarly other relevant structures are populated or loaded from metadata files. The ***inverse_vocab_word_dict*** is the inverse mapping of the word to it's index in thevocabulary for constant time access. One of the structures is also the ***term_document_frequency*** which is every vocabulary term's document frequency.

In [64]:
database_lnc = np.load("database_lnc.npy")
N = database_lnc.shape[0]
vocabulary_dict = pickle.load(open("vocabulary_dict.pkl", "rb"))
vocabulary_keys = list(vocabulary_dict.keys())
inverse_vocab_word_dict = {k: v for v, k in enumerate(vocabulary_keys)}
term_document_frequency = np.count_nonzero(database_lnc, axis=0)
doc_titles = pickle.load(open("doc_titles.pkl", "rb"))

### Initiating Sequence and Search Engine Interface Simulation
After running this cell, we are good to go...!

Sample inputs are mentioned in the report attached as well.

In [65]:
query = input("Please Enter the query to be searched: ")
query = query.split()
scoring(query, database_lnc, vocabulary_keys, inverse_vocab_word_dict, term_document_frequency, N, doc_titles)

Please Enter the query to be searched: New York
Nearest word (Did you mean) heuristic is automatically activated ........... 
Spell Check is activated ........ Running Spell Check
Title weighting is activated ....... 

Top Scoring Documents are: 
"Futurama (New York World's Fair)" is at rank 1 Score: 0.29458171078747575
"Buffalo, New York" is at rank 2 Score: 0.27738496007425395
"Bob Frankston" is at rank 3 Score: 0.17549797644379897
"Big Apple" is at rank 4 Score: 0.15692089291020234
"Brooklyn Historic Railway Association" is at rank 5 Score: 0.11939342119534024
"Futurians" is at rank 6 Score: 0.11439808034788743
"BBC News (TV channel)" is at rank 7 Score: 0.11385650527130695
"Fiorello H. La Guardia" is at rank 8 Score: 0.09432771509209387
"Fantasy Games Unlimited" is at rank 9 Score: 0.0942019695489788
"William M. Tweed" is at rank 10 Score: 0.09365293982216155
